In [7]:
import os
import cv2
import numpy as np



In [8]:
# Function to identify and validate image-mask pairs
def validate_data(root_dir):
    image_files = []
    mask_files = []

    # Recursively search for images and masks
    for subdir, _, files in os.walk(root_dir):
        for file in files:
            if file.endswith('.tif'):
                if "_mask" in file.lower():  # Identify mask files by "_mask" suffix
                    mask_files.append(os.path.join(subdir, file))
                else:
                    image_files.append(os.path.join(subdir, file))

    # Match images with their corresponding masks based on naming convention
    valid_data = []
    for image in image_files:
        # Extract the base name of the image (without directory and extension)
        image_base = os.path.splitext(os.path.basename(image))[0]
        
        # Look for the corresponding mask by adding "_mask" before the extension
        corresponding_mask = None
        for mask in mask_files:
            mask_base = os.path.splitext(os.path.basename(mask))[0]
            if mask_base == f"{image_base}_mask":  # Matching based on the "_mask" pattern
                corresponding_mask = mask
                break
        
        if corresponding_mask:
            valid_data.append((image, corresponding_mask))
        else:
            print(f"Warning: No mask found for image {image_base}")

    return valid_data

In [9]:
# Apply CLAHE preprocessing to enhance contrast
def apply_clahe(image):
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    return clahe.apply(image)

In [10]:
# Function to normalize the image
def normalize_image(image):
    return image / 255.0


In [11]:
# Validate and get list of valid image-mask pairs
valid_pairs = validate_data(data_path)

In [ ]:
# Preprocess the valid images and masks
for image_path, mask_path in valid_pairs:
    # Load image and mask
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

    # Apply CLAHE preprocessing
    processed_image = apply_clahe(image)

    # Normalize image
    normalized_image = normalize_image(processed_image)

    # Display the processed image and mask for visual verification
    cv2.imshow("Processed Image", normalized_image)
    cv2.imshow("Mask", mask)
    cv2.waitKey(0)  # Press any key to move to the next pair

cv2.destroyAllWindows()